# 키워드 기반 매칭 프로필 검색

## 실습을 위한 프로필 데이터베이스 만들기

In [1]:
from typing import List
import json

import pandas as pd
from tqdm.auto import tqdm
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

/Users/hyeonjinho/.pyenv/versions/3.11.6/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
model = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.8)

In [3]:
class Profile(BaseModel):
    name: str = Field(description="이름")
    age: int = Field(description="나이, 20~40")
    gender: str = Field(description="성별: '남자' 또는 '여자'")
    job: int = Field(description="직업")
    bio: str = Field(description="자기소개")
    keywords: List[str] = Field(description="profile을 설명하는 키워드들")

In [4]:
class ProfileList(BaseModel):
    profile_list: List[Profile] = Field(description="소개팅 프로필")

In [5]:
parser = JsonOutputParser(pydantic_object=ProfileList)
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"profile_list": {"title": "Profile List", "description": "소개팅 프로필", "type": "array", "items": {"$ref": "#/definitions/Profile"}}}, "required": ["profile_list"], "definitions": {"Profile": {"title": "Profile", "type": "object", "properties": {"name": {"title": "Name", "description": "이름", "type": "string"}, "age": {"title": "Age", "description": "나이, 20~40", "type": "integer"}, "gender": {"title": "Gender", "description": "성별: \'남자\' 또는 \'여자\'", "type": "string"}, "job": {"title": "Job", "description": "직업", "type": "int

In [6]:
example = {"name":"박지민",
           "age":25,
           "gender":"여자",
           "job":"마케터",
           "bio":"안녕하세요. 25살 박지민입니다. 저는 마케터로 일하고 있어요. 새로운 아이디어를 내는 것을 좋아하고, 사람들과 소통하는 것을 즐깁니다. 영화, 독서가 취미입니다.",
           "keywords":["아이디어","소통","영화", "독서"]}
example_text = json.dumps(example, ensure_ascii=False)
example_text

'{"name": "박지민", "age": 25, "gender": "여자", "job": "마케터", "bio": "안녕하세요. 25살 박지민입니다. 저는 마케터로 일하고 있어요. 새로운 아이디어를 내는 것을 좋아하고, 사람들과 소통하는 것을 즐깁니다. 영화, 독서가 취미입니다.", "keywords": ["아이디어", "소통", "영화", "독서"]}'

In [7]:
human_prompt_template = HumanMessagePromptTemplate.from_template(
                            "소개팅 프로필 {number}개 만들어줘. bio를 작성 할 때는 기존 것들과 비슷하지 않고 다양한 형식으로 만들어줘.\n"
                            "# 예시\n"
                            "{example}\n"
                            "{format_instructions}")

prompt = ChatPromptTemplate.from_messages(
    [
        human_prompt_template,
    ])
prompt = prompt.partial(example=example_text, format_instructions=format_instructions)

In [8]:
profile_gen_chain = prompt | model | parser

In [9]:
profile_obj = profile_gen_chain.invoke({"number": 2})
profile_obj['profile_list']

[{'name': '이태준',
  'age': 30,
  'gender': '남자',
  'job': '소프트웨어 엔지니어',
  'bio': '코드와 함께하는 삶을 즐기는 이태준입니다. 문제를 해결하는 것에 큰 보람을 느끼고, 항상 새로운 기술을 배우는 것에 열정적입니다. 여행과 사진 찍기를 좋아하며, 자연 속에서 영감을 받아 새로운 프로젝트를 구상하기도 합니다.',
  'keywords': ['코딩', '문제 해결', '학습', '여행', '사진']},
 {'name': '정은지',
  'age': 27,
  'gender': '여자',
  'job': '디자이너',
  'bio': '세상을 아름답게 만드는 것이 저의 꿈, 정은지입니다. 디테일에 집중하는 디자이너로서, 사용자 경험을 최우선으로 생각합니다. 미술관 방문을 취미로 삼고 있으며, 여가 시간에는 요가와 명상을 통해 내면의 평화를 찾습니다.',
  'keywords': ['디자인', '사용자 경험', '미술관', '요가', '명상']}]

In [10]:
n_iter = 3
profile_list = []
for _ in tqdm(range(n_iter)):
    profile_obj = profile_gen_chain.invoke({"number": 10})
    profile_list.extend(profile_obj['profile_list'])

  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
df = pd.DataFrame(profile_list)

In [12]:
df.to_json("profile_db.jsonl", orient='records', lines=True, force_ascii=False)

In [13]:
profile_obj

{'profile_list': [{'name': '김태영',
   'age': 30,
   'gender': '남자',
   'job': '소프트웨어 엔지니어',
   'bio': '코드 한 줄로 세상을 바꾸는 꿈을 꾸는 개발자입니다. 새로운 기술에 대한 호기심이 많고, 여행을 통해 영감을 얻습니다.',
   'keywords': ['코딩', '여행', '기술']},
  {'name': '이소윤',
   'age': 27,
   'gender': '여자',
   'job': '그래픽 디자이너',
   'bio': '색채와 무늬로 세상에 아름다움을 더하고 싶은 디자이너. 캔버스에 그림을 그리는 것처럼 삶을 아름답게 그려나가고 있습니다.',
   'keywords': ['디자인', '미술', '창조']},
  {'name': '정민수',
   'age': 32,
   'gender': '남자',
   'job': '운동선수',
   'bio': '모든 순간을 열정적으로 살아내는 운동선수입니다. 스포츠를 통해 배운 팀워크와 도전 정신을 중요하게 생각합니다.',
   'keywords': ['운동', '팀워크', '도전']},
  {'name': '류하나',
   'age': 29,
   'gender': '여자',
   'job': '요리사',
   'bio': '맛있는 음식으로 사람들의 마음을 따뜻하게 해주는 요리사입니다. 음식을 통해 전하는 사랑을 믿습니다.',
   'keywords': ['요리', '음식', '사랑']},
  {'name': '박준혁',
   'age': 34,
   'gender': '남자',
   'job': '자영업자',
   'bio': '새로운 아이디어로 끊임없이 도전하는 창업가. 성공과 실패를 넘어서는 것이 진정한 배움이라 믿습니다.',
   'keywords': ['창업', '도전', '배움']},
  {'name': '최윤아',
   'age': 26,
   'gender': '여자',
   'job': '사진작가',
   'b